# Urban Sound Classification

## Dataset: [UrbanSound8K](https://www.kaggle.com/chrisfilo/urbansound8k) 
    8732 labeled sound excerpts

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Pandas
import pandas as pd

# Scikit learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.utils import class_weight

# Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical


# Audio
import librosa
import librosa.display

# Plot
import matplotlib.pyplot as plt

# Utility
import glob
import numpy as np
from tqdm import tqdm
import IPython.display as ipd
from timeit import default_timer as timer


In [ ]:
# upload the data
classes = pd.read_csv('../input/urbansound8k/UrbanSound8K.csv')

In [ ]:
classes_code = {v:k for k,v in zip(classes["class"], classes["classID"])}
# A dictionnary doesn't accept double values
print(classes_code)

In [ ]:
# Building our dataset
dataset = list()
for folder in glob.glob('../input/urbansound8k/**'):
    list_sounds_path = glob.glob(folder+"//**")
    for elm in list_sounds_path:
        label = elm.split(".wav")[0][-5]
        if label == '-':
            label = elm.split(".wav")[0][-6]
        dataset.append({"file_name" : elm,
                       "label" : label})
        pass
    pass
pass
dataset = pd.DataFrame(dataset)
dataset = shuffle(dataset, random_state = 42)

In [ ]:
dataset.head()

In [ ]:
train, test = train_test_split(dataset, test_size = 0.2, random_state = 42)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
    return mfccs

In [ ]:
x_train = list()
y_train = list()
for i in tqdm(range(len(train))):
    x_train.append(extract_feature(train.file_name.iloc[i]))
    y_train.append(train.label.iloc[i])
    pass
pass

In [ ]:
x_test = list()
y_test = list()
for i in tqdm(range(len(test))):
    x_test.append(extract_feature(test.file_name.iloc[i]))
    y_test.append(test.label.iloc[i])
    pass
pass

In [ ]:
print("x_train",len(x_train),"y_train", len(y_train))

In [ ]:
print("x_train",len(x_train),"y_train", len(y_train))

In [ ]:
X_train, Y_train = list(), list()
for x,y in zip(x_train,y_train):
    if x.shape == (40, 173):
        X_train.append(x)
        Y_train.append(y)
        pass
    pass
pass

In [ ]:
X_test, Y_test = list(), list()
for x,y in zip(x_test,y_test):
    if x.shape == (40, 173):
        X_test.append(x)
        Y_test.append(y)
        pass
    pass
pass

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
%%time
encoder = LabelEncoder()
encoder.fit(Y_train)

Y_train = encoder.transform(Y_train)
Y_test = encoder.transform(Y_test)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
Y_train1 = to_categorical(Y_train)
Y_test1 = to_categorical(Y_test)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(40, 173, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(GlobalAveragePooling2D())

model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


In [ ]:
history = model.fit(X_train, Y_train1,
              batch_size=128,
              epochs=300,
              validation_data=(X_test, Y_test1),
              shuffle=True)

In [ ]:
plt.figure(figsize=[14,10])
plt.subplot(211)
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
# Accuracy Curves
plt.figure(figsize=[14,10])
plt.subplot(212)
plt.plot(history.history['accuracy'],'r',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
model_name = "urban_classification.h5"
model.save(model_name)
print('Saved trained model at %s ' % model_name)

In [ ]:
predictions = model.predict(X_test, verbose=1)

In [ ]:
print(np.argmax(predictions[0]))
print(Y_test[0])

In [ ]:
scores = model.evaluate(X_test, Y_test1, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
print(X_train.shape)

In [ ]:
X_train2 = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
Y_train3 = to_categorical(Y_train)
Y_test3 = to_categorical(Y_test)

In [ ]:
import keras
from keras.layers import GRU

input_shape = (40, 173)

model = keras.models.Sequential()
model.add(GRU(1025, input_shape=input_shape))
model.add(Dense(512, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(10, activation = "softmax"))
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()

In [ ]:
history = model.fit(X_train2, Y_train3,
              batch_size=173,
              epochs=100,
              validation_data=(X_test2, Y_test3),
              shuffle=True)

In [ ]:
plt.figure(figsize=[14,10])
plt.subplot(211)
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
# Accuracy Curves
plt.figure(figsize=[14,10])
plt.subplot(212)
plt.plot(history.history['accuracy'],'r',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
scores = model.evaluate(X_test2, Y_test3, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])